In [175]:
# MLP - with using paper Bengio et al.2003

In [176]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [177]:
# read in all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [178]:
len(words)

29681

In [179]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [180]:
# build the dataset

block_size = 3 # context length: how many characters to use to predict the next one?
X, Y = [], []
for w in words: # [:5]

  # print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join([itos[i] for i in context]), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [181]:
X.shape, X.dtype, Y.shape, Y.dtype 

(torch.Size([212725, 3]), torch.int64, torch.Size([212725]), torch.int64)

In [182]:
# Converting the above code into a function

def build_dataset(words):
  block_size = 3 # context length: how many characters to use to predict the next one
  X, Y = [], []
  for w in words:

    # print(w)
    context = [0] * block_size
    for ch in w + '.':
      X.append(context)
      Y.append(ix)
      # print(''.join([itos[i] for i in context]), '---->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:]) # 10%

torch.Size([170379, 3]) torch.Size([170379])
torch.Size([21222, 3]) torch.Size([21222])
torch.Size([21124, 3]) torch.Size([21124])


In [183]:
len(words)

29681

In [184]:
n1

23744

In [185]:
n2

26712

In [186]:
n2 - n1

2968

In [187]:
len(words) - n2

2969

In [188]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [189]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [190]:
C = torch.randn(27, 2) # each 27 characters is represented by a 2D embedding

In [191]:
C

tensor([[-1.8534, -0.4068],
        [ 0.5352, -1.6474],
        [-2.1217, -0.0860],
        [-0.6625, -0.1120],
        [ 1.4715,  1.9571],
        [-0.1937, -0.4684],
        [ 1.6381, -0.5527],
        [ 0.5527,  1.0046],
        [ 0.6768, -0.6734],
        [ 0.4261, -0.9327],
        [ 0.1062,  0.7216],
        [-1.7297, -1.4324],
        [ 0.6761,  0.4655],
        [ 0.0048,  0.3789],
        [-0.2102,  0.3964],
        [-0.2872, -0.3891],
        [-2.0022, -0.7070],
        [ 0.9121, -0.1046],
        [-0.7447, -0.5701],
        [-0.4313, -0.3535],
        [ 1.4642, -0.9756],
        [-0.3948, -1.8163],
        [-0.2753,  0.3362],
        [-1.1952, -0.9131],
        [-0.8811,  0.1634],
        [ 0.7803,  0.0805],
        [ 0.5614,  2.1519]])

In [192]:
C[5]

tensor([-0.1937, -0.4684])

In [193]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C # one hot vector

tensor([-0.1937, -0.4684])

In [194]:
C[[5,6,7]] # indexing with list

tensor([[-0.1937, -0.4684],
        [ 1.6381, -0.5527],
        [ 0.5527,  1.0046]])

In [195]:
C[torch.tensor([5,6,7])] # indexing with tensor

tensor([[-0.1937, -0.4684],
        [ 1.6381, -0.5527],
        [ 0.5527,  1.0046]])

In [196]:
C[X].shape

torch.Size([212725, 3, 2])

In [197]:
X[13,2] # 13th example, 2nd character

tensor(1)

In [198]:
C[X[13,2]] # embedding of the 13th example, 2nd character

tensor([ 0.5352, -1.6474])

In [199]:
C[1]

tensor([ 0.5352, -1.6474])

In [200]:
emb = C[X]
emb.shape

torch.Size([212725, 3, 2])

In [201]:
# Constructing the hidden layer
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [202]:
h = emb.view(-1, 6) @ W1 + b1

In [203]:
h

tensor([[-2.0890, -7.9114, 10.7474,  ..., -1.3549,  0.1520, -8.5666],
        [-1.5446, -5.1050,  8.4997,  ..., -1.2745,  2.9149, -0.8455],
        [-4.6651, -5.8156,  4.7812,  ..., -0.3017,  7.0756, -2.4895],
        ...,
        [ 5.1452,  4.1186, -6.1267,  ...,  1.7317, -0.4979,  4.5610],
        [ 3.0741,  6.7322, -4.8757,  ...,  0.1303,  0.2120, -1.8144],
        [ 3.7521,  0.9668, -2.9543,  ..., -1.8094, -1.0583, -0.0192]])

In [204]:
# torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape # first, second, third characters of the first example concatenated

In [205]:
# torch.cat(torch.unbind(emb, 1), 1).shape # split the tensor along the second dimension

In [206]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [207]:
a.shape

torch.Size([18])

In [208]:
a.view(2, 9) # view as 2x9 matrix

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [209]:
a.view(9, 2)

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17]])

In [210]:
a.view(3, 3, 2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [211]:
a.storage() # memory address

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [212]:
emb.shape

torch.Size([212725, 3, 2])

In [213]:
# emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

In [214]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [215]:
h

tensor([[-0.9698, -1.0000,  1.0000,  ..., -0.8752,  0.1508, -1.0000],
        [-0.9129, -0.9999,  1.0000,  ..., -0.8550,  0.9941, -0.6887],
        [-0.9998, -1.0000,  0.9999,  ..., -0.2928,  1.0000, -0.9863],
        ...,
        [ 0.9999,  0.9995, -1.0000,  ...,  0.9393, -0.4604,  0.9998],
        [ 0.9957,  1.0000, -0.9999,  ...,  0.1295,  0.2089, -0.9483],
        [ 0.9989,  0.7473, -0.9946,  ..., -0.9478, -0.7850, -0.0192]])

In [216]:
h.shape

torch.Size([212725, 100])

In [217]:
(emb.view(-1, 6) @ W1).shape

torch.Size([212725, 100])

In [218]:
b1.shape

torch.Size([100])

In [219]:
# 32, 100
# broadcasting to 100
# 1 , 100

In [220]:
# Creating the final layer (output layer)
W2 = torch.randn(100, 27)
b2 = torch.randn(27)

In [221]:
logits = h @ W2 + b2

In [222]:
logits.shape

torch.Size([212725, 27])

In [223]:
counts = logits.exp()

In [224]:
prob = counts / counts.sum(-1, keepdim=True)

In [225]:
prob.shape

torch.Size([212725, 27])

In [226]:
prob

tensor([[2.1418e-05, 2.5747e-06, 2.4741e-03,  ..., 4.6982e-10, 3.0663e-07,
         1.8508e-06],
        [1.2659e-07, 1.8981e-08, 3.3634e-09,  ..., 6.2836e-06, 1.5867e-10,
         4.8086e-04],
        [3.0487e-09, 1.3044e-04, 6.8108e-07,  ..., 2.3791e-02, 3.8081e-09,
         1.7835e-05],
        ...,
        [1.3193e-06, 2.2135e-08, 5.2653e-11,  ..., 8.6223e-11, 3.9051e-04,
         8.7195e-09],
        [2.1540e-05, 4.5450e-12, 7.6124e-05,  ..., 7.9294e-10, 7.9628e-01,
         2.7153e-13],
        [9.3199e-07, 7.4203e-02, 3.6436e-06,  ..., 1.7779e-09, 4.0224e-02,
         6.8946e-12]])

In [227]:
prob[0].sum() # normalized

tensor(1.0000)

In [228]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [229]:
torch.arange(36)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])

In [230]:
# prob[torch.arange(36), Y] # probabilities of the true characters

In [231]:
# loss = -prob[torch.arange(36), Y].log().mean() # cross entropy loss
# loss

In [232]:
# ----------------- Summary -----------------

In [233]:
Xtr.shape, Ytr.shape

(torch.Size([170379, 3]), torch.Size([170379]))

In [234]:
g = torch.Generator().manual_seed(2147483647) # seed for reproducibility
C = torch.randn((27, 10), generator=g) # character embeddings
W1 = torch.randn((30, 200), generator=g)  # hidden layer
b1 = torch.randn(200, generator=g) # bias
W2 = torch.randn(200, 27, generator=g)  # output layer
b2 = torch.randn(27, generator=g) # bias
parameters = [C, W1, b1, W2, b2] # all parameters

In [235]:
sum(p.nelement() for p in parameters) # total number of parameters

11897

In [236]:
# Enabling gradient computation
for p in parameters:
  p.requires_grad = True

In [237]:
# finding out the learning rate
lre = torch.linspace(-3, 0, 1000) # 1000 points between 0.001 and 1
lrs = 10 ** lre # learning rates
# lrs

In [238]:
lri = []
lossi = []
step = []

In [239]:
# Training loop
for i in range(50000): # iterations

# mini-batch construction
  ix = torch.randint(0, Xtr.shape[0], (36,)) # 36 random indices from the dataset
# Forward pass

  emb = C[Xtr[ix]] # embedding 
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # hidden layer
  logits = h @ W2 + b2 # output layer
# counts = logits.exp() # unnormalized probabilities
# prob = counts / counts.sum(1, keepdim=True) # normalized probabilities
# loss = -prob[torch.arange(36), Y].log().mean() # cross entropy loss
#  but never use the above code in practice as it is not efficient and numerically unstable
  loss = F.cross_entropy(logits, Ytr[ix]) # cross entropy loss
  # print(loss.item())


  # Backward pass
  for p in parameters:
    p.grad = None

  loss.backward()

  # Update
  # lr = lrs[i]
  lr = 0.01
  for p in parameters:
    p.data += -lr * p.grad # learning rate e.g. -1 (lr)

  # track stats
  # lri.append(lre[i])
  step.append(i)
  lossi.append(loss.log10().item())  

In [240]:
print(loss.item())  

2.0265558760002023e-06


In [241]:
# plt.plot(lri, lossi)
# plt.xscale('log')

In [242]:
# F.cross_entropy(logits, Y) # PyTorch's implementation of cross entropy we can put the cross entropy directly in this code with the help of Pytorch directly in this code which will be more efficient for forward pass and backward pass and numerical stability.

In [243]:
logits.max(1) # max in python is a function that returns two values: the maximum value and its index

torch.return_types.max(
values=tensor([30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286,
        30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286,
        30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286,
        30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286, 30.6286,
        30.6286, 30.6286, 30.6286, 30.6286], grad_fn=<MaxBackward0>),
indices=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [244]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [245]:
# creating minibatches
torch.randint(0, X.shape[0], (36,)) # random integers between 0 and 4

tensor([143353, 130774, 171731, 124893, 180625,  62974, 174274,  19097, 193049,
         66477,  34611, 166457,  94864,  69560,  51841, 208913, 169818,  60051,
         15449, 204467, 163286,  97528, 196664, 127293,  62891, 179840, 193482,
        135105,  80905,  17885,  21186, 106171,  52004, 164486, 139079, 112662])

In [246]:
# evaluation

emb = C[Xdev] # embedding 
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # hidden layer
logits = h @ W2 + b2 # output layer
loss = F.cross_entropy(logits, Ydev) # cross entropy loss
loss


tensor(2.0266e-06, grad_fn=<NllLossBackward0>)

In [247]:
# loss on training set
emb = C[Xtr] # embedding
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # hidden layer
logits = h @ W2 + b2 # output layer
loss = F.cross_entropy(logits, Ytr) # cross entropy loss
loss

tensor(2.0266e-06, grad_fn=<NllLossBackward0>)

In [248]:
# plt.figure(figsize=(8, 8))
# plt.scatter(C[:, 0].data, C[:, 1].data, s=200) # x and y coordinates of the embeddings
# for i in range(C.shape[0]):
  # plt.text(C[i, 0].item(), C[i, 1].item(), itos[i], ha='center', va='center', color='white')
# plt.grid('minor')

In [249]:
# Training split, dev / validation split, test split
# Training split: 80% of the data
# Dev split: 10% of the data
# Test split: 10% of the data

In [ ]:
# sampling from the model
g = torch.Generator().manual_seed(2147483647 + 10) # seed for reproducibility

for _ in range(20):

  out = []
  context = [0] * block_size # initialize with all dots
  while True:
    emb = C[torch.tensor([context])] # embedding
    h = torch.tanh(emb.view(1, -1) @ W1 + b1) # hidden layer
    logits = h @ W2 + b2 # output layer
    probs = F.softmax(logits, dim=1) # probabilities
    ix = torch.multinomial(probs, num_samples=1, generator=g).item() # sample
    context = context[1:] + [ix] # update context
    out.append(ix) # store index
    if ix == 0:
      break

  print(''.join(itos[i] for i in out))

djdyhdyrhguyozvwucovwthslrhcgfccamhjryhcyysewgmtiojyhguyozvyyhjkdokhdnesexjkcynsrqhs.
djdyhvzkhjkdixjyrnuhjryhcyysewgwydiejoyhshodydiejoyhshvyyhjkdokhdneshciyfvyrhguyozvyyhjkdokhdnesewgmtiofvwovsvwidtesjrhguyozvwucovwthsewgmtiojyhguyozvwths.
dzdyxzvkhhjkdokhdnejkyoadiddadiddadiddadiddadiddadiddadiddxshvyyhjkdixjkcynsrqhs.
djdyhdyrzucovwidtesjrhguyozvwths.
djdyhddrzshvyyhjkdixjyrnuhddrewaesyxshvyyhjkyixfkcoyhshojyhguyozvyycoewidigdyxzvkhhjkdoewiekkoovwthslrhcgfyckyfkjryhcyryhcyysovyyhjkdokhdnesewgmtiofvwthslkdaadjdyhdyrhgcbyhshodydiejoyhshojyhguyozvwucovwths.
djdyhddrzvwucovwths.
djdyhddrewaejkhgkbdnrzccvnhguyozvwtesjrhgiddadiddadiddadiddaddrzvwthslrhcgfccamhjkdokhdnesewgmtiofvwaejy.
djdyhshojyhguyrrhshodydiejoyhshwjkhgkbdnrzccvnhguyozvwucovwthslrfucovwthsjkhgkbdnrzccvnhguyozvyycoesyxshvyyhdyrzgthxjyrnuxjnhguyozvwuhjryhcyysewgmtiofvwovsmcjkhgkbdnesewgmtiojyhguyozvyycjyigdyxzvkhhjkdokhdnesexjkcynsrqhs.
djdyhddrzvwucovwthsloduhadiddadiddadiddxshvyyhjkdixjyrhguyozvwths.
djdyhdyrhgigdyxshvy

In [252]:
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
